In [ ]:
import numpy as np
import networkx as nx
import pulp

In [ ]:
def generateGraph(n, density, generator = nx.erdos_renyi_graph):
    g = generator(n, density)
    return g

def getAdjMatrix(graph):
    return nx.adjacency_matrix(graph).todense()

def verifyMDS(graph, dominating_set):
    return nx.algorithms.dominating.is_dominating_set(graph, dominating_set)

In [ ]:
def generateGreedySolution(graph):
    return nx.algorithms.dominating.dominating_set(graph)

def generateExactSolutions(graph, num_solutions = 3, show_messages=True):
    prob = pulp.LpProblem("Minimum Dominating Set", pulp.LpMinimize)
    y = pulp.LpVariable.dicts("y", graph.nodes(), cat=pulp.LpBinary)

    prob += pulp.lpSum(y)
    for v in graph.nodes():
        prob += (y[v] + pulp.lpSum([y[u] for u in graph.neighbors(v)]) >= 1)
        
    solutions = []

    for i in range(num_solutions):
        prob.solve(pulp.apis.PULP_CBC_CMD(msg=show_messages))
        
        selected_vars = []
        for var in y:
             if y[var].value() != 0:
                selected_vars.append(var)
        
        solutions.append(selected_vars)
        
        # Add a new constraint to prevent same solution from being found again
        prob += (pulp.lpSum([y[var] for var in selected_vars]) <= len(selected_vars) - 1)
        
        if pulp.LpStatus[prob.status] != 'Optimal':
            break
            
    return solutions

In [ ]:
graph = generateGraph(200, 0.5)
greedy = generateGreedySolution(graph)
solutions = generateExactSolutions(graph, num_solutions = 3)

In [ ]:
def getEdgeProb(nodes, avg_degree):
    return avg_degree/(nodes - 1)

def getAverageDegrees(minimum = 5, maximum = 15, step = 1):
    return np.arange(minimum, maximum + 1, step)

def getGraphSizes(minimum = 200, maximum = 500, step = 5):
    return np.arange(minimum, maximum + 1, step)

# len(getGraphSizes(500, 1000, 10)) for large range

test = nx.erdos_renyi_graph(100, 10/99)
degrees = [degree[1] for degree in test.degree()]
print(sum(degrees)/len(degrees))

In [ ]:
import scipy.io as sio


graph_data = {
    'adj': getAdjMatrix(graph),
    'greedy': greedy,
    'exact': solutions
}

In [ ]:
degrees = [item[1] for item in list(graph.degree())]
degrees

In [ ]:
greedy

In [ ]:
solutions

In [ ]:
print(f"{1:05}")

In [ ]:
nx.adjacency_matrix(graph).todense().tolist()